In [1]:
from bs4 import BeautifulSoup
from datetime import time
from datetime import datetime
from datetime import timedelta
from collections import Counter
import json
import requests
import csv
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import concurrent.futures
import io
import re

In [3]:
Crossfit = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Crossfit_Men.csv')
Crossfit.head()

/var/folders/xm/kc32pwjj5fj5xqf43yb4dz340000gn/T/ipykernel_5705/3297696159.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Crossfit = pd.read_csv('/Users/cdmoseley/dai_files/Capstone/Galvanize_Capstone_Crossfit/Data/Crossfit_Men.csv')


,Athlete ID,Gender,Competitor Name,Country Name,Region Name,Affiliate Name,Height (in),Age (Years),Weight (lbs),Overall Rank,...,Grace (time),Grace (sec),Helen (time),Helen (sec),L1 Benchmark (time),L1 Benchmark (sec),Run 5k (time),Run 5k (sec),Sprint 400m (time),Sprint 400m (sec)
0,10942,M,Everett Sloan,Canada,North America East,CrossFit Bytown,67.0,45.0,200.0,68985.0,...,1:47,107.0,7:13,433.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10926,M,Kevin Wood,Canada,North America East,CrossFit Moncton,75.0,43.0,196.0,35977.0,...,3:13,193.0,8:16,496.0,NaN,NaN,19:38,1178.0,1:00,60.0
2,10976,M,John Lazaro,United States,North America East,CrossFit Bridgewater,69.0,46.0,180.0,164180.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10977,M,Matthew Haynes,United States,North America West,CrossFit Westwood,71.0,31.0,180.0,165637.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10966,M,Nick DiMatteo,United States,North America East,CrossFit OTG,68.0,48.0,165.0,60027.0,...,NaN,NaN,9:01,541.0,NaN,NaN,21:55,1315.0,NaN,NaN
